# Просьба пока не проверять - дорабатываю

In [24]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (14, 11)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [2]:
df = pd.read_csv('houses.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [33]:
# Заполним пропуски и перекодируем категориальные переменные

In [34]:
for x in df.columns:
    if df[x].dtype == 'object':
        df[x] = df[x].fillna('NA')
    elif df[x].dtype == 'int64':
        df[x] = df[x].fillna(0)
    elif df[x].dtype == 'float64':
        df[x] = df[x].fillna(0)

In [35]:
from sklearn.preprocessing import LabelEncoder

In [36]:
le = LabelEncoder()

In [37]:
le.fit_transform(np.array(df['SaleType'].values))

array([8, 8, 8, ..., 8, 8, 8])

In [38]:
col = [] 
for x in df.columns:
    if df[x].dtype == 'object':
        col.append(x)
        df[x] = le.fit_transform(df[x])

In [39]:
y = df.SalePrice
X = df.drop('SalePrice', axis=1)

In [40]:
num_folds = 10
seed = 7

In [41]:
kfold = KFold(n_splits=num_folds, random_state=seed)

In [42]:
rfr = RandomForestRegressor()
results = cross_val_score(rfr, X, y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0
))

Accuracy: 85.701% (3.736%)


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=7)

In [44]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [45]:
predict = rfr.predict(X_test)

In [46]:
metrics.r2_score(y_test, predict)

0.894286375167488

In [47]:
metrics.explained_variance_score(y_test, predict)

0.8943058654665522

In [31]:
# Выведем важность признаков

In [48]:
i = 0
for x in X.columns:
    print(x, '', rfr.feature_importances_[i])
    i+=1

Id  0.002400831428208919
MSSubClass  0.0017915184869465226
MSZoning  0.0018631445695669539
LotFrontage  0.002503214304818062
LotArea  0.012897941757887282
Street  3.6842566001699675e-06
Alley  0.0005738331142990163
LotShape  0.0005996724908939666
LandContour  0.00253485508103741
Utilities  0.0
LotConfig  0.0003239703733723878
LandSlope  0.0015999561657643498
Neighborhood  0.005509497894945564
Condition1  0.0002562968245622409
Condition2  0.0009701162095195604
BldgType  0.00043126723000153
HouseStyle  0.0009765344577480177
OverallQual  0.5302211022667274
OverallCond  0.003411758003369738
YearBuilt  0.015767213191414147
YearRemodAdd  0.006183869442776752
RoofStyle  0.0007761669890426239
RoofMatl  0.0005473178175002249
Exterior1st  0.0009492030418341877
Exterior2nd  0.001671465335962503
MasVnrType  0.0004833594806134114
MasVnrArea  0.002759700607632607
ExterQual  0.009849479659588326
ExterCond  0.0004145459728751806
Foundation  0.00020901576472715273
BsmtQual  0.006595471431873706
BsmtCon